In [1]:
import numpy as np
import pandas as pd
import gc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task


'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


/home/zhenghao/.conda/envs/light_new/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zhenghao/.conda/envs/light_new/lib/python3.9/site-packages/lightautoml/ml_algo/dl_model.py:42: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/home/zhenghao/.conda/envs/light_new/lib/python3.9/site-packages/lightautoml/text/embed.py:22: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/home/zhenghao/.conda/envs/light_new/lib/python3.9/site-packages/lightautoml/text/dl_transformers.py:25: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")


In [2]:
train = pd.read_csv('/home/zhenghao/kaggle/train.csv')

In [3]:
X_train, X_val = train_test_split(train, test_size=0.2, random_state=42, shuffle=True, stratify=train.Response)

In [4]:
task = Task('binary')

In [5]:
automl = TabularAutoML(
    task=task,  # 指定任务类型
    timeout=600 * 3600,  # 设定训练时间上限为600小时
    cpu_limit=24,  # 使用的CPU核心数量限制为12
    gpu_ids='0',  # 使用第一个GPU
    general_params={"use_algos": [['linear_l2', 'lgb', 'lgb_tuned', 'cb']]},  # 让系统自动选择合适的算法
    reader_params={'n_jobs': 12, 'cv': 5, 'random_state': 42, 'advanced_roles': True}  # 数据读取参数
)

out_of_fold_predictions = automl.fit_predict(
    X_train, valid_data=X_val,  # 训练集和验证集
    roles={
        'target': 'Response',  # 目标列
        'drop': ['id'],  # 丢弃的列
    }, 
    verbose=4  # 显示详细的训练信息
)

[17:10:26] Stdout logging level is DEBUG.
[17:10:26] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[17:10:26] Task: binary

[17:10:26] Start automl preset with listed constraints:
[17:10:26] - time: 2160000.00 seconds
[17:10:26] - CPU: 24 cores
[17:10:26] - memory: 16 GB

[17:10:26] Train data shape: (9203838, 12)



/home/zhenghao/.conda/envs/light_new/lib/python3.9/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


[17:10:39] Feats was rejected during automatic roles guess: []
[17:10:40] Layer 1 train process start. Time left 2159985.92 secs
[17:18:27] Linear model: C = 1e-05 score = 0.8728472677415857
[17:21:11] Linear model: C = 5e-05 score = 0.8742297510889349
[17:21:22] Linear model: C = 0.0001 score = 0.8742298760575264
[17:24:58] Linear model: C = 0.0005 score = 0.8746943820109827
[17:25:02] Linear model: C = 0.001 score = 0.8746943820109827
[17:25:06] Linear model: C = 0.005 score = 0.8746943820109827
[17:25:07] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[17:25:07] Time left 2159119.22 secs

[17:25:13] Training until validation scores don't improve for 100 rounds
[17:25:34] [100]	valid's auc: 0.878627
[17:25:55] [200]	valid's auc: 0.88
[17:26:15] [300]	valid's auc: 0.880321
[17:26:34] [400]	valid's auc: 0.88049
[17:26:53] [500]	valid's auc: 0.880594
[17:27:12] [600]	valid's auc: 0.880683
[17:27:30] [700]	valid's auc: 0.880743
[17:27:48] [800]	valid's auc: 0.880772
[17:28:

In [6]:
del train, X_train, X_val
gc.collect()

527

In [7]:
test = pd.read_csv('/home/zhenghao/kaggle/test.csv')
ss = pd.read_csv('/home/zhenghao/kaggle/sample_submission.csv')

In [8]:
pred = automl.predict(test).data[:,  0]

ss.Response = pred
ss.to_csv('LightAutoML_2.csv', index=None)
ss

,id,Response
0,11504798,0.004403
1,11504799,0.640371
2,11504800,0.239501
3,11504801,0.000070
4,11504802,0.305886
...,...,...
7669861,19174659,0.217761
7669862,19174660,0.000158
7669863,19174661,0.000202
7669864,19174662,0.566881
